In [25]:
### IMPORTS ###
from mmap import mmap, ACCESS_READ
from struct import unpack
from math import sqrt
from glob import glob 
from os import chdir, listdir, makedirs
from os.path import splitext, isdir, exists
import numpy as np
import time
import pandas as pd

In [26]:
# b'\x00\x01\xc4\xb9\xff\xcb\xfb\xe4\x00\x83\x0b'
def unpacking_v2_format_hig(row):
 
    list_values = []
    index = (unpack('>I', bytes(row[0:4]))[0]) 
    acc_x = (unpack('>h', bytes(row[4:6]))[0]) / 1024
    acc_y = (unpack('>h', bytes(row[6:8]))[0]) / 1024
    acc_z = (unpack('>h', bytes(row[8:10]))[0]) / 1024

    gyr_x = (unpack('>h', bytes(row[10:12]))[0]) / 1024
    gyr_y = (unpack('>h', bytes(row[12:14]))[0]) / 1024
    gyr_z = (unpack('>h', bytes(row[14:16]))[0]) / 1024

    #index = (index / 6000)
    #list_values.append(index)
    acc_x = float(acc_x) 
    list_values.append(acc_x)
    acc_y = acc_y 
    list_values.append(acc_y)
    acc_z = acc_z 
    list_values.append(acc_z)
    amag = round(sqrt(pow(acc_x, 2) + pow(acc_y, 2) + pow(acc_z, 2)),2)
    list_values.append(amag)

    list_values.append(gyr_x)
    list_values.append(gyr_y)
    list_values.append(gyr_z)
    
    return list_values

def read_row(mm,hig=1):
    count = 0
    while True:
        count += 1
        if(hig==1):
            row = mm.read(17) # packet length
            if not len(row) == 17: 
                break 
        yield row 

def get_results_v2_format(hig_filename):
    complete_results = []    
    if hig_filename:
        print("getting results for", hig_filename)
        with open(hig_filename,'rb') as file:
            mm = mmap(file.fileno(), 0, access=ACCESS_READ)
            results = [unpacking_v2_format_hig(row) for row in read_row(mm)]
            complete_results.append(results)
            mm.close()
            file.close() 
        
    print("completed conversion",flush=True)
    return complete_results

In [33]:
#hig_filename = "data/mc_5_reps_test_1.HIG"
#hig_filename = "../data/TKA_data/op/OST-0010ser13190358.OST"
#hig_filename = "data/mc_test_3_ml_60bpm.HIG"
#hig_filename = "data/lc_vag_test_1_60bpm.HIG"

converted_data = get_results_v2_format(hig_filename)
print(converted_data[0][0:10])

getting results for ../data/TKA_data/op/OST-0010ser13190358.OST
completed conversion
[[-0.21484375, -0.9189453125, 0.314453125, 0.99, 0.1513671875, 0.03125, -0.4072265625], [-0.2158203125, -0.9296875, 0.296875, 1.0, 0.2138671875, -0.1259765625, -0.4609375], [-0.2177734375, -0.9248046875, 0.3056640625, 1.0, 0.2958984375, -0.044921875, -0.212890625], [-0.228515625, -0.953125, 0.306640625, 1.03, 0.390625, -0.11328125, -0.2978515625], [-0.2060546875, -0.9501953125, 0.3037109375, 1.02, 0.2138671875, 0.09375, -0.400390625], [-0.19921875, -0.9375, 0.302734375, 1.01, 0.220703125, -0.0126953125, -0.4609375], [-0.22265625, -0.9345703125, 0.2998046875, 1.01, 0.251953125, -0.0634765625, -0.2919921875], [-0.2138671875, -0.9248046875, 0.318359375, 1.0, -0.0556640625, -0.1328125, 0.296875], [-0.2158203125, -0.9462890625, 0.3037109375, 1.02, -0.14453125, -0.056640625, 0.3330078125], [-0.240234375, -0.9501953125, 0.2998046875, 1.02, 0.0634765625, -0.107421875, -0.103515625]]


In [34]:
# write to csv
columns = ['x', 'y', 'z', 'mag', 'gyr_x', 'gyr_y', 'gyr_z']

# Create the DataFrame
df = pd.DataFrame(converted_data[0], columns=columns)
print(df.head(5))

df.to_csv("../data/TKA_data/op/tka_op_4.csv", index=False)

          x         y         z   mag     gyr_x     gyr_y     gyr_z
0 -0.214844 -0.918945  0.314453  0.99  0.151367  0.031250 -0.407227
1 -0.215820 -0.929688  0.296875  1.00  0.213867 -0.125977 -0.460938
2 -0.217773 -0.924805  0.305664  1.00  0.295898 -0.044922 -0.212891
3 -0.228516 -0.953125  0.306641  1.03  0.390625 -0.113281 -0.297852
4 -0.206055 -0.950195  0.303711  1.02  0.213867  0.093750 -0.400391
